In [2]:
import pandas as pd
from textblob import TextBlob
import csv
import re
import time
import string
import nltk
from sklearn.metrics import classification_report, confusion_matrix
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
df = pd.read_csv('/content/feedback_limpio.csv')
df[['Review_Text']].head(3)

,Review_Text
0,Absolutely wonderful - silky and sexy and comf...
1,Love this dress!its sooo pretty.i happened to ...
2,I had such high hopes for this dress and reall...


In [4]:
def clean_text(Review_Text):
  Review_Text = re.sub(r'htttps?:\/\/.*[\r\n]*', '', Review_Text)
  Review_Text = re.sub(r'#', '', Review_Text)
  Review_Text = re.sub(r'@[A-Za-z0-9]+', '', Review_Text)
  return Review_Text

In [5]:
df['clean_review'] = df['Review_Text'].apply(clean_text)

In [6]:
df['clean_review']

,clean_review
0,Absolutely wonderful - silky and sexy and comf...
1,Love this dress!its sooo pretty.i happened to ...
2,I had such high hopes for this dress and reall...
3,"I love, love, love this jumpsuit. its fun, fli..."
4,This shirt is very flattering to all due to th...
...,...
22634,I was surprised at the positive reviews for th...
22635,So i wasnt sure about ordering this skirt beca...
22636,I was very happy to snag this dress at such a ...
22637,"It reminds me of maternity clothes. soft, stre..."


In [7]:
def get_polarity(Review_Text):
  analysis = TextBlob(Review_Text)
  if Review_Text != '':


    polarity = analysis.sentiment.polarity


    return polarity



In [8]:
df['polarity'] = df['Review_Text'].apply(get_polarity)

In [9]:
df['polarity'].head(4)

,polarity
0,0.633333
1,0.332500
2,0.073675
3,0.550000


In [10]:
df.head(3)

,Transaction_ID,Clothing_ID,Age,Title,Review_Text,Rating,Recommended_IND,Positive_Feedback_Count,Division_Name,Department_Name,Class_Name,clean_review,polarity
0,100001,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Blouses\n,Absolutely wonderful - silky and sexy and comf...,0.633333
1,100002,1080,34,NaN,Love this dress!its sooo pretty.i happened to ...,5,1,4,General,Dresses,Blouses\n,Love this dress!its sooo pretty.i happened to ...,0.332500
2,100003,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Blouses\n,I had such high hopes for this dress and reall...,0.073675


In [11]:
df[['clean_review', 'polarity', 'Rating']]

,clean_review,polarity,Rating
0,Absolutely wonderful - silky and sexy and comf...,0.633333,4
1,Love this dress!its sooo pretty.i happened to ...,0.332500,5
2,I had such high hopes for this dress and reall...,0.073675,3
3,"I love, love, love this jumpsuit. its fun, fli...",0.550000,5
4,This shirt is very flattering to all due to th...,0.512891,5
...,...,...,...
22634,I was surprised at the positive reviews for th...,-0.078788,5
22635,So i wasnt sure about ordering this skirt beca...,0.211136,5
22636,I was very happy to snag this dress at such a ...,0.552667,3
22637,"It reminds me of maternity clothes. soft, stre...",0.091667,3


In [12]:
df[['clean_review', 'polarity', 'Rating']].to_csv('resultado.csv', index=False)

In [13]:
from google.colab import files
files.download('resultado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
TextBlob('wants').sentiment

Sentiment(polarity=0.2, subjectivity=0.1)

In [15]:
def categorize_polarity(polarity):
  if polarity >= 0.5:
    return 5  # Very positive
  elif polarity >= 0.1:
    return 4  # Positive
  elif polarity >= -0.1:
    return 3  # Neutral
  elif polarity >= -0.5:
    return 2  # Negative
  else:
    return 1  # Very negative

# Apply the categorization function to the 'polarity' column
df['polarity_category'] = df['polarity'].apply(categorize_polarity)

# Now use the categorized polarity for evaluation
print(classification_report(df['Rating'], df['polarity_category']))
print(confusion_matrix(df['Rating'], df['polarity_category']))

              precision    recall  f1-score   support

           1       0.18      0.00      0.00       815
           2       0.18      0.05      0.08      1524
           3       0.17      0.22      0.20      2801
           4       0.22      0.75      0.34      4890
           5       0.68      0.10      0.17     12609

    accuracy                           0.25     22639
   macro avg       0.29      0.22      0.16     22639
weighted avg       0.47      0.25      0.20     22639

[[   2   48  223  500   42]
 [   2   76  400  972   74]
 [   3   77  627 1964  130]
 [   1   82  799 3672  336]
 [   3  149 1536 9694 1227]]
